In [1]:
!python --version
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local

!conda init
!conda install -q python=3.9
!conda create -q --name py39 python=3.9
!source  activate py39
!python --version
# Torch - Torchvision - IceVision - IceData - MMDetection - YOLOv5 - EfficientDet Installation
!wget https://raw.githubusercontent.com/airctic/icevision/master/icevision_install.sh

# Choose your installation target: cuda11 or cuda10 or cpu
!bash icevision_install.sh cuda11 master


Python 3.10.12
--2024-06-30 18:49:37--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146836934 (140M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 140.03M   178MB/s    in 0.8s    

2024-06-30 18:49:38 (178 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [146836934/146836934]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify th

In [2]:
!pip install -q kaggle
!kaggle datasets download -q -d tarunbisht11/yolo-animal-detection-small
!unzip yolo-animal-detection-small.zip  -d data
!rm yolo-animal-detection-small.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 9.0 MB/s eta 0:00:00
Dataset URL: https://www.kaggle.com/datasets/tarunbisht11/yolo-animal-detection-small
License(s): Attribution 4.0 International (CC BY 4.0)
Archive:  yolo-animal-detection-small.zip
  inflating: data/test.csv           
  inflating: data/test.record        
  inflating: data/train.csv          
  inflating: data/train.record       
  inflating: data/yolo-animal-detection-small/test/cats_000.jpg  
  inflating: data/yolo-animal-detection-small/test/cats_000.xml  
  inflating: data/yolo-animal-detection-small/test/cats_007.jpg  
  inflating: data/yolo-animal-detection-small/test/cats_007.xml  
  inflating: data/yolo-animal-detection-small/test/cats_050.jpg  
  inflating: data/yolo-animal-detection-small/test/cats_0

#Overwrite utils.py



In [ ]:
%%writefile /usr/local/lib/python3.9/site-packages/icevision/visualize/utils.py
__all__ = [
    "draw_label",
    "bbox_polygon",
    "draw_mask",
    "as_rgb_tuple",
    "get_default_font",
    "rand_cmap",
]

from icevision.imports import *
from icevision.utils import *
from matplotlib import patches
from PIL import Image, ImageFont, ImageDraw
import PIL


def draw_label(ax, x, y, name, color, fontsize=18):
    ax.text(
        x + 1,
        y - 2,
        name,
        fontsize=fontsize,
        color="white",
        va="bottom",
        bbox=dict(facecolor=color, edgecolor=color, pad=2, alpha=0.9),
    )


def bbox_polygon(bbox):
    bx, by, bw, bh = bbox.xywh
    poly = np.array([[bx, by], [bx, by + bh], [bx + bw, by + bh], [bx + bw, by]])
    return patches.Polygon(poly)


def draw_mask(ax, mask, color):
    color_mask = np.ones((*mask.shape, 3)) * color
    ax.imshow(np.dstack((color_mask, mask * 0.5)))
    ax.contour(mask, colors=[color_mask[0, 0, :]], alpha=0.4)


def as_rgb_tuple(x: Union[np.ndarray, tuple, list, str]) -> tuple:
    "Convert np RGB values -> tuple for PIL compatibility"
    if isinstance(x, (np.ndarray, tuple, list)):
        if not len(x) == 3:
            raise ValueError(f"Expected 3 (RGB) numbers, got {len(x)}")
        if isinstance(x, np.ndarray):
            return tuple(x.astype(np.int32))
        elif isinstance(x, tuple):
            return x
        elif isinstance(x, list):
            return tuple(x)
    elif isinstance(x, str):
        return PIL.ImageColor.getrgb(x)
    else:
        raise ValueError(f"Expected {{np.ndarray|list|tuple}}, got {type(x)}")


def get_default_font() -> str:
    import requests

    font_dir = get_root_dir() / "fonts"
    font_dir.mkdir(exist_ok=True)

    font_file = font_dir / "SpaceGrotesk-Medium.ttf"
    if not font_file.exists():
        URL = "https://raw.githubusercontent.com/airctic/storage/master/SpaceGrotesk-Medium.ttf"
        logger.info(
            "Downloading default `.ttf` font file - SpaceGrotesk-Medium.ttf from {} to {}",
            URL,
            font_file,
        )
        font_file.write_bytes(requests.get(URL).content)
    return str(font_file)


# Generate random colormap from https://stackoverflow.com/questions/14720331/how-to-generate-random-colors-in-matplotlib/32520273#32520273
def rand_cmap(
    nlabels,
    type="bright",
    first_color_black=True,
    last_color_black=False,
    verbose=False,
):
    """
    Creates a random colormap to be used together with matplotlib. Useful for segmentation tasks
    :param nlabels: Number of labels (size of colormap)
    :param type: 'bright' for strong colors, 'soft' for pastel colors
    :param first_color_black: Option to use first color as black, True or False
    :param last_color_black: Option to use last color as black, True or False
    :param verbose: Prints the number of labels and shows the colormap. True or False
    :return: colormap for matplotlib
    """
    from matplotlib.colors import LinearSegmentedColormap
    import colorsys

    np.random.seed(49)

    if type not in ("bright", "soft"):
        print('Please choose "bright" or "soft" for type')
        return

    if verbose:
        print("Number of labels: " + str(nlabels))

    # Generate color map for bright colors, based on hsv
    if type == "bright":
        randHSVcolors = [
            (
                np.random.uniform(low=0.0, high=1),
                np.random.uniform(low=0.2, high=1),
                np.random.uniform(low=0.9, high=1),
            )
            for i in range(nlabels)
        ]

        # Convert HSV list to RGB
        randRGBcolors = []
        for HSVcolor in randHSVcolors:
            randRGBcolors.append(
                colorsys.hsv_to_rgb(HSVcolor[0], HSVcolor[1], HSVcolor[2])
            )

        if first_color_black:
            randRGBcolors[0] = [0, 0, 0]

        if last_color_black:
            randRGBcolors[-1] = [0, 0, 0]

        random_colormap = LinearSegmentedColormap.from_list(
            "new_map", randRGBcolors, N=nlabels
        )

    # Generate soft pastel colors, by limiting the RGB spectrum
    if type == "soft":
        low = 0.6
        high = 0.95
        randRGBcolors = [
            (
                np.random.uniform(low=low, high=high),
                np.random.uniform(low=low, high=high),
                np.random.uniform(low=low, high=high),
            )
            for i in range(nlabels)
        ]

        if first_color_black:
            randRGBcolors[0] = [0, 0, 0]

        if last_color_black:
            randRGBcolors[-1] = [0, 0, 0]
        random_colormap = LinearSegmentedColormap.from_list(
            "new_map", randRGBcolors, N=nlabels
        )

    # Display colorbar
    if verbose:
        from matplotlib import colors, colorbar
        from matplotlib import pyplot as plt

        fig, ax = plt.subplots(1, 1, figsize=(15, 0.5))

        bounds = np.linspace(0, nlabels, nlabels + 1)
        norm = colors.BoundaryNorm(bounds, nlabels)

        cb = colorbar.ColorbarBase(
            ax,
            cmap=random_colormap,
            norm=norm,
            spacing="proportional",
            ticks=None,
            boundaries=bounds,
            format="%1i",
            orientation="horizontal",
        )

    return random_colormap


#Train model


In [9]:
%%writefile train.py
import fastai.vision
import fastai.vision.all
from icevision.all import *
from icevision.models import *
import os
from icevision.core import BaseRecord
import pandas as pd
import fastai

row_traincsv_data_path = os.path.join(r"/content/data/train.csv")
df = pd.read_csv(row_traincsv_data_path)

df.rename(columns={"class":"label"},inplace=True)

train_annotation_path = os.path.join(r"/content/data/yolo-animal-detection-small","train_annotation.csv")
df.to_csv(train_annotation_path,index=False)
df = pd.read_csv(train_annotation_path)

template_record = ObjectDetectionRecord()

class AmDetec(Parser):
    def __init__(self,template_record):
        super().__init__(template_record=template_record)

        self.df = pd.read_csv(train_annotation_path)
        self.class_map = ClassMap(list(self.df["label"].unique())) # {"background":0,"cat":1,"dog":2}

    def __iter__(self) -> Any:
        for o in self.df.itertuples():
            yield o   #จะคืนค่าทีละแถวแทนที่จะคืนค่าทั้งหมดในคราวเดียว

    def __len__(self) -> int:
        return len(self.df)

    def record_id(self,o) -> Hashable:
        return o.filename
    def parse_fields(self, o, record, is_new):
        if is_new:
            record.set_filepath(os.path.join(r"/content/data/yolo-animal-detection-small","train",o.filename)) #setไฟล์pathเพื่อ train
            record.set_img_size(ImgSize(width=o.width,height=o.height))
            record.detection.set_class_map(self.class_map)
        record.detection.add_bboxes([BBox.from_xyxy(o.xmin,o.ymin,o.xmax,o.ymax)])
        record.detection.add_labels([o.label])

parser = AmDetec(template_record=template_record)
train_record,val_record = parser.parse()
model_type = models.mmdet.retinanet
backbone = model_type.backbones.resnet50_fpn_1x

train_tramform = tfms.A.Adapter([*tfms.A.aug_tfms(size=224,presize=512),tfms.A.Normalize()])
valid_tramform = tfms.A.Adapter([*tfms.A.resize_and_pad(224),tfms.A.Normalize])


train_set = Dataset(train_record,train_tramform)
val_set = Dataset(val_record,valid_tramform)

train_dl = model_type.train_dl(train_set,batch_size=8,num_workers=4,shuffle =True)
val_dl = model_type.train_dl(train_set,batch_size=8,num_workers=4,shuffle =False)



model = model_type.model(backbone=backbone(pretrained=True),num_classes=len(parser.class_map)) # model_type คือ object detection model ของเราชื่ออะไร ส่่วน classification เป็นตัส backbone


metrics = [COCOMetric(metric_type=COCOMetricType.bbox)] #mAP score

#ใช้ fast AI ในการ train model

csv_logger = fastai.vision.all.CSVLogger(fname='training_log.csv')

# เพิ่ม COCOMetric ใน callbacks เพื่อบันทึกค่า AP และ AR ในแต่ละ epoch




learner = model_type.fastai.learner(dls=[train_dl,val_dl],model=model,metrics=metrics,cbs=csv_logger)

#lr = learner.lr_find()
#print(lr)
#learner.fine_tune(5,1e-4,freeze_epochs=1)
learner.fine_tune(50, 7e-5, freeze_epochs=1)
m = ClassMap(list(df['label'].unique()))
m.get_classes()

checkpoint_path = "monkeycatdog_det.pth"
save_icevision_checkpoint(
    model,
    model_name='mmdet.retinanet',
    backbone_name='resnet50_fpn_1x',
    classes=m.get_classes(),
    img_size=224,
    filename=checkpoint_path,
    meta={"icevision_version": "0.12.0"}
)

Overwriting train.py


In [10]:
!python train.py

INFO     - The mmdet config folder already exists. No need to downloaded it. Path : /root/.icevision/mmdetection_configs/mmdetection_configs-2.20.1/configs | icevision.models.mmdet.download_configs:download_mmdet_configs:17
INFO     - The mmseg config folder already exists. No need to downloaded it. Path : /root/.icevision/mmsegmentation_configs/mmsegmentation_configs-0.17.0/configs | icevision.models.mmseg.download_configs:download_mmseg_configs:19
100% 1309/1309 [00:00<00:00, 12864.28it/s]
INFO     - Autofixing records | icevision.parsers.parser:parse:122
100% 469/469 [00:00<00:00, 18707.30it/s]
/usr/local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoi

#Test model

In [11]:
%%writefile test.py
from icevision.all import *
from icevision.models import *
from icevision.core import BaseRecord
import pandas as pd
from PIL import Image, ImageDraw
import glob
import matplotlib.pyplot as plt
row_traincsv_data_path = os.path.join(r"/content/data/train.csv")
df = pd.read_csv(row_traincsv_data_path)

df.rename(columns={"class":"label"},inplace=True)

train_annotation_path = os.path.join(r"/content/data/yolo-animal-detection-small","train_annotation.csv")
df.to_csv(train_annotation_path,index=False)
df = pd.read_csv(train_annotation_path)

template_record = ObjectDetectionRecord()

class AmDetec(Parser):
    def __init__(self,template_record):
        super().__init__(template_record=template_record)

        self.df = pd.read_csv(train_annotation_path)
        self.class_map = ClassMap(list(self.df["label"].unique())) # {"background":0,"cat":1,"dog":2}

    def __iter__(self) -> Any:
        for o in self.df.itertuples():
            yield o   #จะคืนค่าทีละแถวแทนที่จะคืนค่าทั้งหมดในคราวเดียว

    def __len__(self) -> int:
        return len(self.df)

    def record_id(self,o) -> Hashable:
        return o.filename
    def parse_fields(self, o, record, is_new):
        if is_new:
            record.set_filepath(os.path.join(r"/content/data/yolo-animal-detection-small","train",o.filename)) #setไฟล์pathเพื่อ train
            record.set_img_size(ImgSize(width=o.width,height=o.height))
            record.detection.set_class_map(self.class_map)
        record.detection.add_bboxes([BBox.from_xyxy(o.xmin,o.ymin,o.xmax,o.ymax)])
        record.detection.add_labels([o.label])

parser = AmDetec(template_record=template_record)
train_record,val_record = parser.parse()
model_type = models.mmdet.retinanet
backbone = model_type.backbones.resnet50_fpn_1x

train_tramform = tfms.A.Adapter([*tfms.A.aug_tfms(size=224,presize=512),tfms.A.Normalize()])
valid_tramform = tfms.A.Adapter([*tfms.A.resize_and_pad(224),tfms.A.Normalize()])


train_set = Dataset(train_record,train_tramform)
val_set = Dataset(val_record,valid_tramform)

model_loaded = model_from_checkpoint("/content/monkeycatdog_det.pth")
model_type = model_loaded["model_type"]
backbone = model_loaded["backbone"]
class_map = model_loaded["class_map"]
img_size = model_loaded["img_size"]

model_type.show_results(model_loaded["model"], val_set, detection_threshold=0.4)

infer_dl = model_type.infer_dl(val_set, batch_size=4, shuffle=False) #อนุมานให้้ valset ทำเป็น dataloader แล้วเอาไป ทำการสร้างbbox ผ่าน

#retinanet infer_dl = model_type.infer_dl(val_set, batch_size=4, shuffle=False): ทำการสร้าง DataLoader (infer_dl) จากชุดข้อมูลที่ต้องการทำนาย (val_set) โดยที่จำนวน batch คือ 4 และไม่มีการสลับข้อมูล (shuffle=False)
# โดย DataLoader นี้จะถูกใช้ในการส่งภาพไปให้กับโมเดล RetinaNet เพื่อทำการตรวจจับวัตถุ (object detection) บนภาพเหล่านั้นๆ และคืนค่า bounding box ที่โมเดลคาดการณ์ได้ในแต่ละภาพ.

preds = model_type.predict_from_dl(model_loaded["model"], infer_dl, keep_images=True) #นำ รูปใน bbox ไป classification ใน resnet 50

#preds = model_type.predict_from_dl(model_loaded["model"], infer_dl, keep_images=True): นำ DataLoader (infer_dl) ที่ได้มาใช้ในการทำนายด้วยโมเดลที่โหลดมาจาก checkpoint (model_loaded["model"])
# โดยทำการตรวจจับวัตถุในภาพที่อยู่ใน bounding box ที่ได้แล้ว และทำการจำแนกประเภทของวัตถุใน bounding box โดยใช้ ResNet-50 ที่เรียนรู้มาแล้วจากภาพที่อยู่ใน bounding box นั้นๆ ซึ่งผลลัพธ์ที่ได้จะเป็นรายการของ predictions พร้อมกับภาพที่ถูกจัดประเภทไว้ในแต่ละ bounding box.

#พารามิเตอร์ keep_images=True ที่ถูกใช้ในฟังก์ชัน model_type.predict_from_dl() มีหน้าที่บอกให้ฟังก์ชันเก็บภาพที่ผ่านการประมวลผล (inference) โดยโมเดลไว้ด้วย นั่นหมายความว่า เมื่อเราทำการทำนาย (inferencing) บนภาพใน DataLoader แล้ว
# ไม่เพียงแค่ส่งผลลัพธ์ที่เป็นการจัดประเภท (classification) หรือตรวจจับวัตถุ (object detection) กลับมาเท่านั้น แต่ยังเก็บภาพต้นฉบับที่ผ่านการประมวลผลของโมเดลไว้ด้วยด้วย
#การทำให้ฟังก์ชัน predict_from_dl() เก็บภาพต้นฉบับนี้เอาไว้เป็นไปได้เพื่อใช้ในการแสดงผลหรือการตรวจสอบผลลัพธ์ทีหลัง หรือในการวิเคราะห์เพิ่มเติมโดยไม่ต้องใช้ภาพต้นฉบับใหม่เสมอไป ซึ่งเป็นประโยชน์ในการลดการใช้ทรัพยากรและเพิ่มประสิทธิภาพของการทำงานโดยรวม

## สมมติว่า preds เป็นผลลัพธ์ที่ได้จาก predict_from_dl()
#for result in preds:
    # ดูคีย์ทั้งหมดใน dictionary ของผลลัพธ์
#    print(result.keys())

plt.figure(figsize=(12, 12))
imgA = show_preds(preds=preds[:4])
plt.axis('off')
plt.savefig("/content/imgRESULT1.jpg")
plt.close()

img = Image.open((glob.glob(os.path.join(r"/content/data/yolo-animal-detection-small/train","*.jpg")))[4])
pred_dict  = model_type.end2end_detect(img, valid_tramform, model_loaded["model"], class_map=class_map, detection_threshold=0.5)
imgg = pred_dict['img']
imgg.save("/content/imgRESULT2.jpg")

print(pred_dict.keys())

for i in pred_dict.keys():
  print(pred_dict[i])



Writing test.py


In [12]:
!python test.py

INFO     - The mmdet config folder already exists. No need to downloaded it. Path : /root/.icevision/mmdetection_configs/mmdetection_configs-2.20.1/configs | icevision.models.mmdet.download_configs:download_mmdet_configs:17
INFO     - The mmseg config folder already exists. No need to downloaded it. Path : /root/.icevision/mmsegmentation_configs/mmsegmentation_configs-0.17.0/configs | icevision.models.mmseg.download_configs:download_mmseg_configs:19
100% 1309/1309 [00:00<00:00, 12512.49it/s]
INFO     - Autofixing records | icevision.parsers.parser:parse:122
100% 469/469 [00:00<00:00, 18509.97it/s]
load checkpoint from local path: /content/monkeycatdog_det.pth
/usr/local/lib/python3.9/site-packages/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  warnings.warn(
2024-06-30 19:39:46,547 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024

In [14]:
!pip list >requirements.txt